# Análisis de oferta

Solución para hacer análisis de oferta del modelo de abastecimiento estratégico.

In [1]:
import pandas as pd
import numpy as np
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Paquetes de consulta a traves de Socrata

from sodapy import Socrata
identificador_columnas=pd.read_csv('datos/identificador_columnas.csv')
valor_columnas={'SECOP I':'API_DA_S_I','SECOP II':'API_DA_S_II','TVEC':'API_DA_S_TVEC'}
identificador_columnas=identificador_columnas[['Unificado','API_DA_S_I', 'API_DA_S_II']]
identificador_columnas=identificador_columnas.dropna()
identificador_columnas=identificador_columnas.set_index('Unificado')
identificador_columnas.drop('Fuente',axis=0,inplace=True)
identificador_datos={'SECOP I':'f789-7hwg','SECOP II':'jbjy-vk9h','TVEC':'rgxm-mmea'}
valor_columnas={'SECOP I':'API_DA_S_I','SECOP II':'API_DA_S_II','TVEC':'API_DA_S_TVEC'}
### Definición de cliente para Socrata

cliente=Socrata('www.datos.gov.co',None,timeout=1000000)


In [2]:
identificador_columnas

,API_DA_S_I,API_DA_S_II
Unificado,,
ID Contrato,numero_de_contrato,id_contrato
ID Proceso,uid,proceso_de_compra
ID entidad Plataforma,c_digo_de_la_entidad,codigo_entidad
Entidad,nombre_entidad,nombre_entidad
NIT Entidad,nit_de_la_entidad,nit_entidad
Orden,nivel_entidad,orden
Modalidad,modalidad_de_contratacion,modalidad_de_contratacion
Estado,estado_del_proceso,estado_contrato
Descripción proceso,detalle_del_objeto_a_contratar,descripcion_del_proceso


In [4]:
Queries_list=[]
results=[]
for i in ['SECOP I','SECOP II']:
    query_SECOP="""
    SELECT 
        {2} as entidad,
        {3} as departamento_entidad,
        {4} as orden,
        {5} as nit_entidad,
        count({0}) as cantidad,
        sum({1}) as suma_valor_contrato
    Group by 
        {2},{3},{4},{5}
    LIMIT 
        1000000
    """.format(identificador_columnas[valor_columnas[i]]['ID Contrato'],identificador_columnas[valor_columnas[i]]['Valor del contrato'],
    identificador_columnas[valor_columnas[i]]['Entidad'],identificador_columnas[valor_columnas[i]]['Departamento Entidad'],
    identificador_columnas[valor_columnas[i]]['Orden'],identificador_columnas[valor_columnas[i]]['NIT Entidad'])
    Queries_list.append(query_SECOP)
    results_detalle = cliente.get(identificador_datos[i], query=query_SECOP)
    results.append(results_detalle)


In [ ]:
results_dataframe=pd.concat([pd.DataFrame(results[0]),pd.DataFrame(results[1])])
results_dataframe.reset_index(inplace=True)
results_dataframe.drop('index',axis=1,inplace=True)
results_dataframe['suma_valor_contrato']=results_dataframe['suma_valor_contrato'].astype(float)
results_dataframe['cantidad']=results_dataframe['cantidad'].astype(int)
results_dataframe['orden']=results_dataframe['orden'].str.upper()
results_dataframe['departamento_entidad']=results_dataframe['departamento_entidad'].replace('Distrito Capital de Bogotá','Bogotá DC')\
    .replace('San Andrés Providencia y Santa Catalina','San Andrés, Providencia y Santa Catalina').replace('Norte De Santander','Norte de Santander')

In [ ]:
dptos={}
for i in results_dataframe['departamento_entidad'].unique():
    dptos.update({i:[i]})
dptos['Bogotá DC']=['Bogotá DC','Distrito Capital de Bogotá']
dptos['San Andrés, Providencia y Santa Catalina']=['San Andrés, Providencia y Santa Catalina','San Andrés Providencia y Santa Catalina']
dptos['Norte de Santander']=['Norte de Santander','Norte De Santander']
 


In [ ]:
def consulta_departamento(departamento):
    results_detalle=[]
    for i in ['SECOP I','SECOP II']:
        queries_list=[]
        results_detalle_dpto=[]
        for j in dptos[departamento]:
            query_secop_tot=""" Select
                {0} as id_contrato,
                {1} as id_proceso,
                {2} as valor_contrato,
                {3} as entidad,
                {4} as departamento_entidad,
                {5} as orden,
                {6} as modalidad,
                {7} as tipo_contrato,
                {8} as codigo_unspsc,
                {9} as objeto_contrato,
                {10} as descripcion_proceso,
                {11} as nombre_proveedor,
                {12} as tipo_documento_proveedor,
                {13} as numero_documento_proveedor,
                {14} as fecha_inicio,
                {15} as fecha_fin,
                {16} as fecha_firma,
                {17} as url

            where
                {4}='{18}'
            limit
                10000000
            """.format(identificador_columnas[valor_columnas[i]]['ID Contrato'],identificador_columnas[valor_columnas[i]]['ID Proceso'],
            identificador_columnas[valor_columnas[i]]['Valor del contrato'],
            identificador_columnas[valor_columnas[i]]['Entidad'],identificador_columnas[valor_columnas[i]]['Departamento Entidad'],
            identificador_columnas[valor_columnas[i]]['Orden'],identificador_columnas[valor_columnas[i]]['Modalidad'],
            identificador_columnas[valor_columnas[i]]['Tipo de contrato'],identificador_columnas[valor_columnas[i]]['UNSPSC'],
            identificador_columnas[valor_columnas[i]]['Objeto Contrato'],identificador_columnas[valor_columnas[i]]['Descripción proceso'],
            identificador_columnas[valor_columnas[i]]['Nombre Proveedor'],identificador_columnas[valor_columnas[i]]['Tipo de documento proveedor'],
            identificador_columnas[valor_columnas[i]]['Documento Proveedor'],identificador_columnas[valor_columnas[i]]['Inicio de contrato'],
            identificador_columnas[valor_columnas[i]]['Fin de contrato'],identificador_columnas[valor_columnas[i]]['Fecha de Firma'],
            identificador_columnas[valor_columnas[i]]['URL'],j)
            queries_list.append(query_secop_tot)
            df_temp=pd.DataFrame(cliente.get(identificador_datos[i], query=query_secop_tot))
            df_temp['fuente']=i
            if df_temp.shape[0]>0:
                results_detalle_dpto.append(df_temp)
        if len(results_detalle_dpto)>0:
            results_detalle.append(pd.concat(results_detalle_dpto))
    results_detalle=pd.concat(results_detalle)
    results_detalle.reset_index(inplace=True)
    results_detalle.drop('index',axis=1,inplace=True)
    results_detalle['valor_contrato']=results_detalle['valor_contrato'].astype(float)
    results_detalle['departamento_entidad']=results_detalle['departamento_entidad'].replace('Distrito Capital de Bogotá','Bogotá DC')\
    .replace('San Andrés Providencia y Santa Catalina','San Andrés, Providencia y Santa Catalina').replace('Norte De Santander','Norte de Santander')
      
    return results_detalle
        

In [ ]:
def consulta_datos_departamento(departamento):
    results_detalle=[]
    for i in ['SECOP I','SECOP II']:
        queries_list=[]
        results_detalle_dpto=[]
        for j in dptos[departamento]:
            query_secop_tot=""" Select
                count({0}) as cantidad_contratos,
                count({1}) as cantidad_procesos,
                sum({2}) as suma_valor_contratos,
                {3} as entidad,
                {4} as departamento_entidad,
                {5} as orden,
                {6} as modalidad,
                
            where
                {4}='{7}'
            
            limit
                10000000
            """.format(identificador_columnas[valor_columnas[i]]['ID Contrato'],identificador_columnas[valor_columnas[i]]['ID Proceso'],
            identificador_columnas[valor_columnas[i]]['Valor del contrato'],
            identificador_columnas[valor_columnas[i]]['Entidad'],identificador_columnas[valor_columnas[i]]['Departamento Entidad'],
            identificador_columnas[valor_columnas[i]]['Orden'],identificador_columnas[valor_columnas[i]]['Modalidad'],
            identificador_columnas[valor_columnas[i]]['Tipo de contrato'],identificador_columnas[valor_columnas[i]]['UNSPSC'],
            identificador_columnas[valor_columnas[i]]['Objeto Contrato'],identificador_columnas[valor_columnas[i]]['Descripción proceso'],
            identificador_columnas[valor_columnas[i]]['Nombre Proveedor'],identificador_columnas[valor_columnas[i]]['Tipo de documento proveedor'],
            identificador_columnas[valor_columnas[i]]['Documento Proveedor'],identificador_columnas[valor_columnas[i]]['Inicio de contrato'],
            identificador_columnas[valor_columnas[i]]['Fin de contrato'],identificador_columnas[valor_columnas[i]]['Fecha de Firma'],
            identificador_columnas[valor_columnas[i]]['URL'],j)
            queries_list.append(query_secop_tot)
            df_temp=pd.DataFrame(cliente.get(identificador_datos[i], query=query_secop_tot))
            df_temp['fuente']=i
            if df_temp.shape[0]>0:
                results_detalle_dpto.append(df_temp)
        if len(results_detalle_dpto)>0:
            results_detalle.append(pd.concat(results_detalle_dpto))
    results_detalle=pd.concat(results_detalle)
    results_detalle.reset_index(inplace=True)
    results_detalle.drop('index',axis=1,inplace=True)
    results_detalle['valor_contrato']=results_detalle['valor_contrato'].astype(float)
    results_detalle['departamento_entidad']=results_detalle['departamento_entidad'].replace('Distrito Capital de Bogotá','Bogotá DC')\
    .replace('San Andrés Providencia y Santa Catalina','San Andrés, Providencia y Santa Catalina').replace('Norte De Santander','Norte de Santander')
      
    return results_detalle

,id_contrato,id_proceso,valor_contrato,entidad,departamento_entidad,orden,modalidad,tipo_contrato,codigo_unspsc,objeto_contrato,descripcion_proceso,nombre_proveedor,tipo_documento_proveedor,numero_documento_proveedor,fecha_inicio,fecha_fin,fecha_firma,url,fuente
0,06818,18-4-8778420-7979584,9910000.0,SAN ANDRÉS PROVIDENCIA Y SANTA CATALINA INSTI...,"San Andrés, Providencia y Santa Catalina",TERRITORIAL,Régimen Especial,Obra,721033,"Servicios de Edificación, Construcción de Inst...",SERVICIOS MANTENIMIENTO A TODO COSTO DE INFRAE...,JONATHAN MANUEL PEREZ MERCADO,Cédula de Ciudadanía,1123626969,2018-12-13T00:00:00.000,2018-12-21T00:00:00.000,2018-12-13T00:00:00.000,{'url': 'https://www.contratos.gov.co/consulta...,SECOP I
1,CD7302021,21-12-12139046-11243285,8060000.0,SAN ANDRÉS PROVIDENCIA Y SANTA CATALINA ALCAL...,"San Andrés, Providencia y Santa Catalina",TERRITORIAL,Contratación Directa (Ley 1150 de 2007),Prestación de Servicios,941215,Organizaciones y Clubes,Prestación de servicio de apoyo a la gestión c...,LAUREANA TAYLOR SMITH,Cédula de Ciudadanía,1120980309,2021-02-02T00:00:00.000,2021-07-02T00:00:00.000,2021-02-02T00:00:00.000,{'url': 'https://www.contratos.gov.co/consulta...,SECOP I
2,CD N 020 DE 2016,16-12-4966870-4553694,11737440.0,SAN ANDRÉS PROVIDENCIA Y SANTA CATALINA ALCAL...,"San Andrés, Providencia y Santa Catalina",TERRITORIAL,Contratación Directa (Ley 1150 de 2007),Prestación de Servicios,801116,"Servicios de Gestion, Servicios Profesionales ...",EL CONTRATISTA SE OBLIGA PARA CON EL MUNICIPIO...,FRANCISCO JOSE HUFFINGTON BRITTON,Cédula de Ciudadanía,8724739,2016-02-01T00:00:00.000,2016-08-01T00:00:00.000,2016-02-01T00:00:00.000,{'url': 'https://www.contratos.gov.co/consulta...,SECOP I
3,No definido,16-12-5968329,NaN,SAN ANDRÉS PROVIDENCIA Y SANTA CATALINA ALCAL...,"San Andrés, Providencia y Santa Catalina",TERRITORIAL,Contratación Directa (Ley 1150 de 2007),Prestación de Servicios,801116,"Servicios de Gestion, Servicios Profesionales ...",SERVICIOS TEMPORALES PERSONALES PARA LABORES D...,No Definido,No Definido,No Definido,NaN,NaN,NaN,{'url': 'https://www.contratos.gov.co/consulta...,SECOP I
4,541 DE 2015,15-12-3736744-3494849,28922634.0,SAN ANDRÉS PROVIDENCIA Y SANTA CATALINA GOBER...,"San Andrés, Providencia y Santa Catalina",TERRITORIAL,Contratación Directa (Ley 1150 de 2007),Prestación de Servicios,N/D,"Servicios de Gestion, Servicios Profesionales ...",Prestar sus servicios profesionales como Biólo...,CRISTINA ANDREA ZAPATA MASPOLI,Cédula de Ciudadanía,40992274,2015-03-27T00:00:00.000,2015-12-27T00:00:00.000,2015-03-27T00:00:00.000,{'url': 'https://www.contratos.gov.co/consulta...,SECOP I
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49081,CO1.PCCNTR.3967483,CO1.BDOS.3205344,8225625.0,SERVICIO NACIONAL DE APRENDIZAJE SENA REGIONAL...,"San Andrés, Providencia y Santa Catalina",Nacional,Contratación directa,Prestación de servicios,V1.80111600,PRESTACIÓN DE SERVICIOS PERSONALES DE CARÁCTER...,PRESTACIÓN DE SERVICIOS PERSONALES DE CARÁCTER...,Dido Dixon illiams Nelson,Cédula de Ciudadanía,18000580,2022-09-01T00:00:00.000,2022-12-17T00:00:00.000,2022-08-30T15:08:33.000,{'url': 'https://community.secop.gov.co/Public...,SECOP II
49082,CO1.PCCNTR.1828972,CO1.BDOS.1441050,10055535.0,GOBERNACIÓN DEL DEPARTAMENTO ARCHIPIELAGO DE S...,"San Andrés, Providencia y Santa Catalina",Territorial,Contratación directa,Prestación de servicios,V1.80111600,Prestación de servicios profesionales como adm...,Prestación de servicios profesionales como adm...,DEVI ESTIGUAR ROMERO HERAZO,Cédula de Ciudadanía,1123627320,2020-09-22T00:00:00.000,2020-12-30T00:00:00.000,2020-09-14T16:09:53.000,{'url': 'https://community.secop.gov.co/Public...,SECOP II
49083,CO1.PCCNTR.2335241,CO1.BDOS.1833624,34065027.0,GOBERNACIÓN DEL DEPARTAMENTO ARCHIPIELAGO DE S...,"San Andrés, Providencia y Santa Catalina",Territorial,Contratación directa,Prestación de servicios,V1.80111600,PRESTACION DE SERVICIOS PROFESIONALES COMO E